In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np

PATH='data/digit-recognizer/'
SZ=28
BATCH_SIZE=128
STEPS = 1000
MODEL_DIR='models/mnist_2dense'

In [3]:
data = pd.read_csv(f'{PATH}train.csv')
display(data.head())
features, labels = data, data.pop('label')
labels.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


0    1
1    0
2    1
3    4
4    0
Name: label, dtype: int64

In [8]:
import matplotlib.pyplot as plt
from skimage.transform import resize
f = np.asarray(features).reshape(-1, 28,28)
resized_features = []
for i, img in enumerate(f):
    image = resize(img, (224, 224), preserve_range=True)
    resized_features.append(image)
    if i % 1000 == 0:
        print(i)

# np.shape(resized_features)
# print(image)
# plt.imshow(image)
# image = np.repeat(image[:, :, np.newaxis], 3, axis=2)
# plt.imshow(image)

(42000, 224, 224)

In [ ]:
features = np.asarray(features, dtype=np.float32)
labels = np.asarray(labels, dtype=np.int64)

In [18]:
from sklearn.model_selection import train_test_split
train_x, val_x, train_y, val_y = train_test_split(features, labels, test_size=0.2)

In [22]:
def cnn_model(features, labels, mode, params):
    input_layer = tf.reshape(features["x"], [-1, SZ, SZ, 1]) # Channel first NCWH
    conv1 = tf.layers.conv2d(input_layer, 32, kernel_size=[5, 5], 
                             padding="same", activation=tf.nn.relu)
    pool1 = tf.layers.max_pooling2d(conv1, pool_size=[2, 2], strides=2)
    conv2 = tf.layers.conv2d(pool1, 64, kernel_size=[5, 5], 
                             padding="same", activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(conv2, pool_size=[2, 2], strides=2)
    
#     conv3 = tf.layers.conv2d(pool2, 128, kernel_size=[5, 5], 
#                              padding="same", activation=tf.nn.relu)
#     pool3 = tf.layers.max_pooling2d(conv3, pool_size=[2, 2], strides=2)
    
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
    
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    dense = tf.layers.dense(inputs=dropout, units=512, activation=tf.nn.relu)
    dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    logits = tf.layers.dense(inputs=dropout, units=10)
    
    """ PREDICT """
    predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)



    """ EVAL """
    # Add evaluation metrics (for EVAL mode)
    
    accuracy = tf.metrics.accuracy(labels=labels,
                                    predictions=predictions["classes"],
                                    name='acc_op')
    metrics={'accuracy': accuracy}
    tf.summary.scalar('accuracy', accuracy[1])

    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(mode, loss=loss, eval_metric_ops=metrics)

    """ TRAIN """
    # Configure the Training Op (for TRAIN mode)
    assert mode == tf.estimator.ModeKeys.TRAIN
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    

In [23]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x = {'x': train_x},
    y = train_y, 
    batch_size = BATCH_SIZE,
    num_epochs=None,
    shuffle=True)

eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": val_x},
    y=val_y,
    num_epochs=1,
    shuffle=False)

predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": test_x},
    y=None,
    num_epochs=1,
    shuffle=False)

In [29]:
classifier = tf.estimator.Estimator(
    cnn_model, 
    model_dir=MODEL_DIR
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'models/mnist_2dense', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f07b43a7320>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models/mnist_2dense/model.ckpt-3000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 3001 into 

In [ ]:
for _ in range(10):
    classifier.train(
        input_fn=train_input_fn,
        steps=STEPS
    )
    
    # Evaluate the model and print results
    eval_results = classifier.evaluate(input_fn=eval_input_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models/mnist_2dense/model.ckpt-4000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 4001 into models/mnist_2dense/model.ckpt.
INFO:tensorflow:loss = 0.081242576, step = 4001
INFO:tensorflow:global_step/sec: 10.6284
INFO:tensorflow:loss = 0.010529525, step = 4101 (9.410 sec)
INFO:tensorflow:global_step/sec: 10.7162
INFO:tensorflow:loss = 0.17078012, step = 4201 (9.332 sec)
INFO:tensorflow:global_step/sec: 10.7376
INFO:tensorflow:loss = 0.06158732, step = 4301 (9.313 sec)
INFO:tensorflow:global_step/sec: 10.6934
INFO:tensorflow:loss = 0.22383046, step = 4401 (9.352 sec)
INFO:tensorflow:global_step/sec: 10.7453
INFO:tensorflow:loss = 0.045566894, step = 4501 (9.306 sec)
INFO:tensorflow:global_step/sec: 10.7354
INFO:tensorfl

In [ ]:
test = pd.read_csv(f'{PATH}test.csv')
test_x = np.asarray(test, dtype=np.float32)

In [ ]:
predictions = classifier.predict(input_fn=predict_input_fn)

In [ ]:
results=[['ImageId', 'Label']]
for i, prediction in enumerate(predictions):
    results.append([i+1, prediction['classes']])
    

In [ ]:
len(results)

In [ ]:
np.savetxt("csv/mnist.csv", np.asarray(results), delimiter=",", fmt="%s")